In [ ]:
import re 
import numpy as np
import pandas as pd

file = 'cudnn.log'

## CONVOLUTION

In [ ]:
blocks = []
count = {}

funcPattern = re.compile(r'I! CuDNN .* function (cudnnConvolutionForward|cudnnConvolutionBackwardData|cudnnConvolutionBackwardFilter)\(\) called:')

with open(file, 'r') as logfile:
    while(True):
        line = logfile.readline()
        if not line:
            break

        # do stuff
        funcInfo = re.match(funcPattern, line)
        if funcInfo:
            # misc: counts of fwd, bwd data and bwd filter calls.
            if funcInfo.group(1) not in count.keys():
                count[funcInfo.group(1)] = 0
            count[funcInfo.group(1)] += 1

            block = line
            line = logfile.readline()
            while(line.strip()):
                block += line
                line = logfile.readline()
            blocks.append(block)

print(len(blocks))
print(count)

In [ ]:
xPattern = re.compile(r'i!\s+dimA: type=int; val=\[(?P<n>\d+),(?P<c>\d+),(?P<h>\d+),(?P<w>\d+)\];')
wPattern = re.compile(r'i!\s+dimA: type=int; val=\[(?P<k>\d+),(?P<i_prime>\d+),(?P<f_h>\d+),(?P<f_w>\d+)\];')
padPattern = re.compile(r'i!\s+padA: type=int; val=\[(?P<pad_h>\d+),(?P<pad_w>\d+)\];')
stridePattern = re.compile(r'i!\s+strideA: type=int; val=\[(?P<stride_h>\d+),(?P<stride_w>\d+)\];')
dilPattern = re.compile(r'i!\s+dilationA: type=int; val=\[(?P<dil_h>\d+),(?P<dil_w>\d+)\];')
groupPattern = re.compile(r'i!\s+groupCount: type=int; val=(?P<groupCount>\d+);')
algoPattern = re.compile(r'i!\s+algo: type=(?:cudnnConvolutionFwdAlgo_t|cudnnConvolutionBwdDataAlgo_t|cudnnConvolutionBwdFilterAlgo_t); val=(?P<algo>[a-zA-Z0-9_ ()]+);')
dtypePattern = re.compile(r'i!\s+dataType: type=cudnnDataType_t; val=(?P<type>[a-zA-Z0-9_ ()]+);')
mathPattern = re.compile(r'i!\s+mathType: type=cudnnMathType_t; val=(?P<type>[a-zA-Z0-9_ ()]+);')


convsInfo = []
for i, block in enumerate(blocks):
    conv = {}
    
    lines = block.split('\n')
    
    ## CUDNN FUNCTION
    funcInfo = re.match(funcPattern, lines[0])
    conv['function'] = funcInfo.group(1)
    
    ## INPUT INFORMATION
    xDescLine = [x for x in range(len(lines)) if 'xDesc' in lines[x]]
    assert len(xDescLine) == 1, "multiple 'xDesc' in block:{}".format(i+1)
    xDescLine = xDescLine[0]
    
    # Note: input information is in the 3rd line after xDesc key
    xInfo = xPattern.match(lines[xDescLine + 3])
    if xInfo:
        conv['n'] = int(xInfo.group('n'))
        conv['c'] = int(xInfo.group('c'))
        conv['h'] = int(xInfo.group('h'))
        conv['w'] = int(xInfo.group('w'))
    else:
        print("ERROR: Input dimensions not found at expected spot for block:{}".format(i+1))
        conv['n'] = conv['c'] = conv['h'] = conv['w'] = -1
        
    ## Filter Information
    wDescLine = [x for x in range(len(lines)) if 'wDesc' in lines[x]]
    assert len(wDescLine) == 1, "multiple 'wDesc' in block:{}".format(i+1)
    wDescLine = wDescLine[0]
    
    # NOTE: filter information is in the 4th line after wDesc key.
    wInfo = wPattern.match(lines[wDescLine + 4])
    if wInfo:
#         conv['inDim'] = int(wInfo.group('i_prime')) # this is actually (inDim/groupCount)
        conv['k'] = int(wInfo.group('k'))
        conv['f_h'] = int(wInfo.group('f_h'))
        conv['f_w'] = int(wInfo.group('f_w'))
    else:
        print("ERROR: Filter dimensions not found at expected spot for block:{}.".format(i+1))
        conv['k'] = conv['f_h'] = conv['f_w'] = -1
    
    ## CONVOLUTION INFO
    convDescLine = [x for x in range(len(lines)) if 'convDesc' in lines[x]]
    assert len(convDescLine) == 1, "multiple 'convDesc' in block:{}".format(i+1)
    convDescLine = convDescLine[0]
    
    # Padding
    # NOTE: padding information is in the 6th line after convDesc key.
    padInfo = padPattern.match(lines[convDescLine + 6])
    if padInfo:
        conv['pad_h'] = int(padInfo.group('pad_h'))
        conv['pad_w'] = int(padInfo.group('pad_w'))
    else:
        print("ERROR: Padding dimensions not found at expected spot for block:{}.".format(i+1))
        conv['pad_h'] = conv['pad_w'] = -1
    
    # Stride
    # NOTE: stride info is in the 7th line after convDesc key.
    strideInfo = stridePattern.match(lines[convDescLine + 7])
    if strideInfo:
        conv['stride_h'] = int(strideInfo.group('stride_h'))
        conv['stride_w'] = int(strideInfo.group('stride_w'))
    else:
        print("ERROR: Padding dimensions not found at expected spot for block:{}.".format(i+1))
        conv['stride_h'] = conv['stride_w'] = -1
    
    # Dilation
    # NOTE: dilation info is in the 8th line after convDesc key.
    dilInfo = dilPattern.match(lines[convDescLine + 8])
    if dilInfo:
        conv['dil_h'] = int(dilInfo.group('dil_h'))
        conv['dil_w'] = int(dilInfo.group('dil_w'))
    else:
        print("ERROR: Dilation dimensions not found at expected spot for block:{}.".format(i+1))
        conv['dil_h'] = conv['dil_w'] = -1
    
    # Group Count
    # NOTE: groupcount info is in the 9th line after convDesc key.
    groupInfo = groupPattern.match(lines[convDescLine + 9])
    if groupInfo:
        conv['group'] = int(groupInfo.group('groupCount'))
    else:
        print("ERROR: Dilation dimensions not found at expected spot for block:{}.".format(i+1))
        conv['group'] = -1
        
    # Data Type
    dtypeInfo = dtypePattern.match(lines[convDescLine + 2])
    if dtypeInfo:
        conv['dataType'] = dtypeInfo.group('type')
    else:
        print("ERROR: DataType not found at expected spot for block:{}".format(i+1))
        conv['dataType'] = "UNKNOWN"

    # Math Type
    mathInfo = mathPattern.match(lines[convDescLine + 3])
    if mathInfo:
        conv['mathType'] = mathInfo.group('type')
    else:
        print("ERROR: MathType not found at expected spot for block:{}".format(i+1))
        conv['mathType'] = "UNKNOWN"
    
    ## ALGORITHM INFO
    algoLine = [x for x in range(len(lines)) if 'algo' in lines[x]]
    assert len(algoLine) == 1, "multiple 'algo' in block:{}".format(i+1)
    algoLine = algoLine[0]
    algoInfo = algoPattern.match(lines[algoLine])
    if algoInfo:
        conv['algo'] = algoInfo.group('algo')
    else:
        print("ERROR: Algorithm information not found at expected spot for block:{}.".format(i+1))
        conv['algo'] = "UNKNOWN"
    
    # Add to list
    convsInfo.append(conv)

# for item in convsInfo:
#     print(item)
print(len(convsInfo))

In [ ]:
df = pd.DataFrame(convsInfo)
print(len(df))
display(df)

In [ ]:
df.to_csv('cudnn-allconvs.csv')

In [ ]:
unique_df = df[df.function == 'cudnnConvolutionForward'][['n','c','h','w','k','f_h','f_w','pad_h','pad_w', \
                                                          'stride_h', 'stride_w','dil_h','dil_w','group']].drop_duplicates()
print(len(unique_df))
display(unique_df)

In [ ]:
# print(df)
distinctConv = unique_df.apply(lambda item: "std::make_tuple({},{},{},{},{},{},{},{},{},{},{},{})".format(item['w'], item['h'], item['c'], item['n'],
                                                                           item['k'], item['f_w'], item['f_h'], item['pad_w'],
                                                                           item['pad_h'], item['stride_w'], item['stride_h'],
                                                                           item['group']), axis=1)
allconvstr = ",\n".join(distinctConv)
print(allconvstr)

In [ ]:
# Save dataframe - to be used in parseDeepBench.ipynb
df.to_pickle('/workspace/resnext101_32x8d-archinfo.pkl')

## BATCHNORM

In [ ]:
blocks = []
with open(file, 'r') as logfile:
    while(True):
        line = logfile.readline()
        if not line:
            break
        # do stuff
        if 'cudnnBatchNormalizationForwardTraining()' in line:
            block = line
            line = logfile.readline()
            while(line.strip()):
                block += line
                line = logfile.readline()
            blocks.append(block)
print(len(blocks))

In [ ]:
bnInfo = []
for i, block in enumerate(blocks):
    bn = {}
    
    lines = block.split('\n')
    
    ## INPUT INFORMATION
    xDescLine = [x for x in range(len(lines)) if 'xDesc' in lines[x]]
    assert len(xDescLine) == 1, "multiple 'xDesc' in block:{}".format(i+1)
    xDescLine = xDescLine[0]
    
    xPattern = re.compile(r'i!\s+dimA: type=int; val=\[(?P<n>\d+),(?P<c>\d+),(?P<h>\d+),(?P<w>\d+)\];')
    # Note: input information is in the 3rd line after xDesc key
    xInfo = xPattern.match(lines[xDescLine + 3])
    if xInfo:
        bn['n'] = int(xInfo.group('n'))
        bn['c'] = int(xInfo.group('c'))
        bn['h'] = int(xInfo.group('h'))
        bn['w'] = int(xInfo.group('w'))
    else:
        print("ERROR: Input dimensions not found at expected spot for block:{}".format(i+1))
        bn['n'] = bn['c'] = bn['h'] = bn['w'] = -1
        
    ## ScaleBiasMeanVar Information
    sbmvLine = [x for x in range(len(lines)) if 'bnScaleBiasMeanVarDesc' in lines[x]]
    assert len(sbmvLine) == 1, "multiple 'xDesc' in block:{}".format(i+1)
    sbmvLine = sbmvLine[0]    

    sbmvPattern = re.compile(r'i!\s+dimA: type=int; val=\[(?P<n>\d+),(?P<c>\d+),(?P<h>\d+),(?P<w>\d+)\];')
    # Note: input information is in the 3rd line after xDesc key
    sbmvInfo = sbmvPattern.match(lines[sbmvLine + 3])
    if sbmvInfo:
        assert int(sbmvInfo.group('n')) == 1, "This is expected to be 1"
        bn['num_features'] = int(sbmvInfo.group('c'))
        assert int(sbmvInfo.group('h')) == 1, "This is expected to be 1"
        assert int(sbmvInfo.group('w')) == 1, "This is expected to be 1"
    else:
        print("ERROR: num_features not found at expected spot for block:{}".format(i+1))
        bn['num_features'] = -1
    
    ## Momentum
    pLine = [x for x in range(len(lines)) if 'exponentialAverageFactor' in lines[x]]
    assert len(pLine) == 1, "multiple 'xDesc' in block:{}".format(i+1)
    pLine = pLine[0]
    
    pPattern = re.compile(r'i!\s+exponentialAverageFactor: type=double; val=(?P<momentum>\d*\.\d+);')
    pInfo = pPattern.match(lines[pLine])
    if pInfo:
        bn['momentum'] = float(pInfo.group('momentum'))
    else:
        print("ERROR: momentum not found at expected spot for block:{}".format(i+1))
        bn['momentum'] = float(-1)
    
    ## Epsilon
    epsLine = [x for x in range(len(lines)) if 'epsilon' in lines[x]]
    assert len(epsLine) == 1, "multiple 'xDesc' in block:{}".format(i+1)
    epsLine = epsLine[0]
    
    epsPattern = re.compile(r'i!\s+epsilon: type=double; val=(?P<epsilon>\d*\.\d+);')
    epsInfo = epsPattern.match(lines[epsLine])
    if epsInfo:
        bn['epsilon'] = float(epsInfo.group('epsilon'))
    else:
        print("ERROR: epsilon not found at expected spot for block:{}".format(i+1))
        bn['epsilon'] = float(-1)
        
    # Add to list
    bnInfo.append(bn)

print(len(bnInfo))
# for item in bnInfo:
#     print(item)

In [ ]:
import pandas as pd
batchnorms = pd.DataFrame(bnInfo)
print(len(batchnorms))

unique_bn = batchnorms.drop_duplicates()
print(len(unique_bn))
print(unique_bn)

In [ ]:
distinctConv = unique_bn.apply(lambda item: "std::make_tuple({},{},{},{},{},{},{})"
                               .format(item['w'].astype(int), 
                                       item['h'].astype(int),
                                       item['c'].astype(int),
                                       item['n'].astype(int), 
                                       item['num_features'].astype(int),
                                       item['momentum'],
                                       item['epsilon']),
                               axis=1)
allconvstr = ",\n".join(distinctConv)
print(allconvstr)